In [1]:
# Program used for studying how to systematically record results by using CSVLogger, exporting model to a json file, exporting 
# weights in Hd5 format, using time stamps, using text files to store stdout etc.
# created by @leopauly (cnlp@leeds.ac.uk)

import numpy
import datetime
import glob
import sys
from PIL import Image  
from scipy import misc
from scipy import ndimage
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle

from keras.datasets import mnist
from keras.datasets import cifar10
from keras.callbacks import History,CSVLogger 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


Using TensorFlow backend.


In [2]:
seed = 7
numpy.random.seed(seed)


In [3]:
#imageFolderPath = './allnew/'
#imagePath = glob.glob(imageFolderPath+'/*.jpg') 
#X = numpy.array([numpy.array((Image.open(imagePath[i]).convert('L')), 'f') for i in range(len(imagePath))] )
#print (X.shape)
#img_rows=32
#img_cols=32
#channel=1


In [4]:
#y_1=numpy.ones(13233)
#y_2=numpy.zeros(7390)
#y=numpy.append(y_1,y_2)
#nb_classes=1
#print (y.shape)

In [5]:
#X,y = shuffle(X,y, random_state=1)
#X,y = shuffle(X,y, random_state=2)
#X,y = shuffle(X,y, random_state=3)
#X,y = shuffle(X,y, random_state=4)

In [6]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [7]:
K.set_image_dim_ordering('th')
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
img_rows=32
img_cols=32
channel=3
nb_classes=10

In [8]:
print (X_train.shape)
print (X_test.shape)

K.set_image_dim_ordering('th')
X_train = X_train.reshape(X_train.shape[0], channel,img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], channel,img_rows, img_cols)
input_shape=(channel,img_rows,img_cols)

print (X_train.shape)
print (X_test.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.
X_test /= 255.



(50000, 3, 32, 32)
(10000, 3, 32, 32)
(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [9]:
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print (y_train.shape)
print (y_test.shape)

(50000, 10)
(10000, 10)


In [10]:
print (y_test)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  1.  0.  0.]]


In [11]:
model = Sequential()
model.add(Convolution2D(16, 3, 3, input_shape=input_shape, activation='relu',border_mode='valid'))
model.add(Dropout(0.5))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='valid'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))


In [12]:
epochs = 10
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 16, 30, 30)    448         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 16, 30, 30)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 28, 28)    4640        dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 32, 28, 28)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [13]:
b_size=128
#csv_logger = CSVLogger('trainingcnn4.log',separator=',', append=True)

start_time=datetime.datetime.now()
print (start_time)

history=model.fit(X_train, y_train, nb_epoch=epochs, 
                  batch_size=b_size, validation_split=0.2, 
                  shuffle='batch')

end_time=datetime.datetime.now()
print (end_time)

2017-02-09 13:53:11.448770
Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 63s - loss: 2.1310 - acc: 0.2061 - val_loss: 2.0905 - val_acc: 0.2850
Epoch 2/10
40000/40000 [==============================] - 50s - loss: 1.9439 - acc: 0.3004 - val_loss: 1.9826 - val_acc: 0.3526
Epoch 3/10
40000/40000 [==============================] - 50s - loss: 1.8457 - acc: 0.3437 - val_loss: 1.9026 - val_acc: 0.3818
Epoch 4/10
40000/40000 [==============================] - 50s - loss: 1.7811 - acc: 0.3649 - val_loss: 1.8594 - val_acc: 0.4040
Epoch 5/10
40000/40000 [==============================] - 50s - loss: 1.7227 - acc: 0.3867 - val_loss: 1.7921 - val_acc: 0.4237
Epoch 6/10
40000/40000 [==============================] - 50s - loss: 1.6718 - acc: 0.4043 - val_loss: 1.7369 - val_acc: 0.4426
Epoch 7/10
40000/40000 [==============================] - 49s - loss: 1.6170 - acc: 0.4214 - val_loss: 1.6848 - val_acc: 0.4624
Epoch 8/10
40000/40000 [===

In [14]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 49.54%


In [ ]:
#storing model to a json file and weights in HDF5 format

model_json = model.to_json()
# saving model
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5 and saving it
model.save_weights("model.txt")
print("Saved model to disk")
 
# load json file and load the model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Saved model to disk
Loaded model from disk


In [ ]:
#attempt to store model and weights to a simple text file
#Preparing documentation

sys.stdout = open('resultslog.txt', 'a')

print("\n",'LeedsNet <Attempt 3> : Results when LeedsNet was trained using MNIST dataset',"\n")
print("\n",'Network architechture',"\n")
print(model.summary())
print("\n",'Results: Training',"\n",history.history,"\n")
print("\n",'Time taken for training',"\n",' Start Time:', start_time,"\n",'End Time:', end_time,"\n")
print("\n","Accuracy on testing data: %.2f%%" % (scores[1]*100),"\n")

sys.stdout.close()